# Step 4: '개선 방안 발상' 및 효과 설명 (ToT 발상 + CoT 효과, RAG 필수 활용)

### 목표

선택된 핵심 문제점을 해결하기 위한 창의적이고 실현 가능한 개선 방안 발상. 이때도 RAG를 적극 활용해서 현실적인 방안 도출

### 수정 내용:

- 개선 방안 발상시, GPT 프롬프트에 **Step 3에서 선정된 '핵심 문제점'의 상세 내용(요약, 근거 포함)과 더불어, 약관 원문 및 관련 외부 참고 자료(벡터 스토어 검색 결과)**를 반드시 포함
- 프롬프트 지침에 "제시된 핵심 문제점과 관련 약관 및 **추가 참고 자료**를 바탕으로 효과적인 개선 방안을 발상하라"고 명시. 특히 "다른 약관에서는 이 문제를 어떻게 해결하고 있는지 **[참고 자료]**를 통해 확인하고 아이디어를 얻으라"고 지시 가능.
- 개선 방안의 '해결 논리 및 기대 효과' (CoT) 부분을 설명할 때, 어떤 개선 방안을 적용하면 -> 약관의 어떤 문구가 어떻게 바뀌고/추가 안내가 제공되며 -> 이게 어떤 논리적 과정을 거쳐 핵심 문제점을 해결하고 -> 어떤 긍정적인 효과(소비자 이해도 증가, 분쟁 감소 등)가 나타나는지, 그리고 이를 뒷받침할 만한 외부 참고 자료의 내용은 없는지 구체적으로 설명하도록 유도
- ToT 발상 단계이므로 temperature는 높게, Self-consistency는 동일 적용.

### ✅ Step 4: 문제점 후보 평가 및 핵심 문제 도출 (최종 정리본)
#### 🎯 목적


*   Step 2~3에서 생성된 문제점 후보들 중 유사한 항목 추가
*   요약끼리 정규화/군집화
*   GPT를 활용하여 핵심 문제점으로 통합
*   최종적으로 실질적인 개선안 도출의 기반이 되는 문제점들을 선별



In [ ]:
from collections import defaultdict

In [1]:
def group_problem_candidates_by_summary(problem_candidates_list, min_length=5):
    """
    문제점 후보들을 '문제점_요약' 기준으로 군집화하고, 빈도순으로 정렬
    """
    grouped = defaultdict(list)
    for p in problem_candidates_list:
        summary = p.get("문제점_요약", "").strip()
        if summary and len(summary) >= min_length:
            grouped[summary].append(p)

    sorted_clusters = sorted(grouped.items(), key=lambda x: len(x[1]), reverse=True)

    print(f"\n✅ Step 4-1: 총 {len(sorted_clusters)}개의 클러스터 생성 완료")
    return sorted_clusters

In [2]:
def evaluate_cluster_with_gpt(summary, group, model="gpt-4o-2024-05-13", temperature=0.3):
    """
    하나의 클러스터(유사 요약 후보 그룹)에 대해 GPT로 핵심 문제점 도출
    """
    candidate_texts = "\n\n".join([
        f"[후보 {i+1}]\n- 문제점 요약: {p.get('문제점_요약')}\n- 근거 및 추론: {p.get('근거_및_추론과정')}\n- 참고 출처: {p.get('참고_자료_출처')}"
        for i, p in enumerate(group)
    ])

    prompt = f"""
당신은 보험 약관 분석 전문가입니다.
다음은 유사한 문제점 후보들의 묶음입니다.
각각은 '문제점 요약', '근거 및 추론', '참고 출처'를 포함합니다.

이들을 **중복 없이 하나의 핵심 문제점**으로 통합해 주세요.

요구 사항:
- 표현은 명료하고 객관적일 것
- 실제 제도 개선 혹은 소비자 보호 관점에서 실질적으로 중요한 관점을 유지
- 통합 사유도 명확히 작성

문제점 후보들:
{candidate_texts}

출력 형식 (JSON):
{{
  "핵심_문제점_요약": "통합된 핵심 문제점 요약",
  "통합_사유": "왜 이들이 하나로 묶일 수 있는지 설명",
  "관련_후보_수": {len(group)}
}}
"""

    try:
        result = request_gpt(
            prompt,
            model=model,
            temperature=temperature,
            type="json_object"
        )
        result["관련_후보_수"] = len(group)
        return result
    except Exception as e:
        print(f"[GPT 오류] 클러스터 통합 실패: {e}")
        return None

In [3]:
def run_step4_extract_core_problems(problem_candidates_list, model="gpt-4o-2024-05-13", top_n=5):
    """
    Step 4 전체 실행: 후보 → 클러스터링 → GPT 통합 → 핵심 문제 도출
    """
    if not problem_candidates_list:
        print("🚨 문제점 후보 리스트가 비어 있습니다.")
        return []

    print(f"\n📊 Step 4 시작: 총 후보 수 = {len(problem_candidates_list)}")

    # 1. 클러스터링
    clustered = group_problem_candidates_by_summary(problem_candidates_list)

    # 2. 상위 N개 클러스터 평가
    final_core_problems = []
    for idx, (summary, group) in enumerate(clustered[:top_n]):
        print(f"\n▶️ 클러스터 {idx+1}: 후보 수={len(group)}, 요약='{summary[:50]}...'")
        result = evaluate_cluster_with_gpt(summary, group, model=model)
        if result:
            final_core_problems.append(result)
            print(f"✅ 핵심 문제 도출: {result['핵심_문제점_요약']}")
        else:
            print("❌ GPT 평가 실패, 생략됨")

    print(f"\n🎯 Step 4 완료: 핵심 문제점 총 {len(final_core_problems)}건 도출됨")
    return final_core_problems